In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#Lets provide some insight into PO.DAAC impact metrics 
#in Elseviers SCOPUS service

In [ ]:
#First lets import the libraries we require
from pprint import pprint
import podaac.podaac as podaac
import podaac.podaac_utils as utils

In [ ]:
#Then we can create instances of the classes we will use
p = podaac.Podaac()
u = utils.PodaacUtils()

In [ ]:
#Print a list of dataset short names
print('\nHeres list_all_available_granule_search_dataset_short_names()')
result = u.list_all_available_granule_search_dataset_short_names()
dsetShortName = [i for i in result]
print(dsetShortName)
print(len(result))

In [ ]:
#dataset_landing_pages = []
#Perform a landing page search on each dataset, note this can take some time as we are querying >600 times
#for ds in dsetShortName:
#    result = p.dataset_search(short_name=ds)
#    #Cache the dataset landing page URL
#    searchStr = 'http://podaac.jpl.nasa.gov/dataset/'
#    for i in result.strip().split():
#        if searchStr in i:
#            dataset_landing_pages.append(str(i))
#print(dataset_landing_pages)

In [ ]:
from bs4 import BeautifulSoup
dois = []
#Perform a doi search on each dataset, note this can take some time as we are querying >600 times
for ds in dsetShortName:
    try:
        result = p.dataset_search(short_name=ds, full='True')
    except Exception:
        print('Error retrieving record for ' + ds)
    soup = BeautifulSoup(result, 'html.parser')
    doi_element = soup.find('podaac:datasetdoi', text=True)
    if doi_element is not None:
        doi = doi_element.text #do not include 'https://doi.org/' prefix as it eventually makes search less accurate and increases runtime
        dois.append(doi)
print(dois)

In [ ]:
print(len(dois))

In [ ]:
import json
import requests
data = []
#Using Elsevier's Scopus Search, lets see if we can 
#retreieve any information from the above dataset dois
for doi in dois:
    url = 'https://api.elsevier.com/content/search/scopus?query=ALL:' + doi + '&APIKey=715b412c00f0b95e918a3e7abe6e6ee4'
    try:
        metadata = requests.get(url)
        status_codes = [404, 400, 503, 408]
        if metadata.status_code in status_codes:
            metadata.raise_for_status()
    except requests.exceptions.HTTPError as error:
        print(error)
        raise
    json_data = json.loads(metadata.text)
    entries = json_data['search-results']['entry']
    for entry in entries:
        if 'error' not in entry:
            print(entry)
            try:
                prism_doi = entry['prism:doi']
            except KeyError:
                prism_doi = '.'               
            data.append(['.', '.', entry['dc:title'], entry['prism:publicationName'], entry['citedby-count'], entry['prism:url'], prism_doi, '.', '.', '.', '.', '.', '.', '.', '.'])

In [ ]:
#Again, using Elsevier's Scopus Search, lets see other podaac.jpl.nasa.gov resource we can retreive.
url = 'https://api.elsevier.com/content/search/scopus?query=ALL:podaac.jpl.nasa.gov&APIKey=715b412c00f0b95e918a3e7abe6e6ee4'
try:
    metadata = requests.get(url)
    status_codes = [404, 400, 503, 408]
    if metadata.status_code in status_codes:
        metadata.raise_for_status()
except requests.exceptions.HTTPError as error:
    print(error)
    raise
json_data = json.loads(metadata.text)
entries = json_data['search-results']['entry']
for entry in entries:
    data.append(['.', '.', entry['dc:title'], entry['prism:publicationName'], entry['citedby-count'], entry['prism:url'], entry['prism:doi'], '.', '.', '.', '.', '.', '.', '.', '.'])

In [ ]:
import pandas as pd
#Simply print the data as a pandas data frame
pd.DataFrame(data, columns=["Dataset Name as Cited in Article", "PERSISTENT_ID", "Manuscript Title", "Journal Source", "Impact factor of the Manuscript", "Article URL", "DOI", "PDF Name", "In-Text", "In-Text 2", "Referenced", "Referenced 2", "In-Text", "Referenced", "Confirmed"])